In [1]:
import pandas as pd
import numpy as np

# Gerar dataset para cada estado

In [2]:
datasetTotal = pd.read_csv('datasetTotal.csv')

In [3]:
listaUF = datasetTotal['UF'].unique()

In [4]:
datasetTotal.head(10)

,Unnamed: 0,Município_x,UF,Idade,Total_Estado,Total_Homem,Total_Mulher,Urbana_Total,Urbana_Homem,Urbana_Mulher,Rural_Total,Rural_Homem,Rural_Mulher,editMun,ID,Município_y
0,0,Alta Floresta D'Oeste (RO),RO,Total,6880,3520,3360,4011,1951,2060,2869,1569,1300,altaflorestadoeste,1,Alta Floresta dOeste
1,1,NaN,RO,15 a 17 anos,1577,830,747,878,438,440,699,392,307,altaflorestadoeste,1,Alta Floresta dOeste
2,2,NaN,RO,18 ou 19 anos,1011,532,479,604,301,303,407,231,176,altaflorestadoeste,1,Alta Floresta dOeste
3,3,NaN,RO,20 a 24 anos,2200,1105,1095,1303,632,671,897,473,424,altaflorestadoeste,1,Alta Floresta dOeste
4,4,NaN,RO,25 a 29 anos,2092,1053,1039,1226,580,646,866,473,393,altaflorestadoeste,1,Alta Floresta dOeste
5,5,Ariquemes (RO),RO,Total,26480,13096,13384,23011,11284,11727,3469,1812,1657,ariquemes,2,Ariquemes
6,6,NaN,RO,15 a 17 anos,5497,2758,2739,4641,2310,2331,856,448,408,ariquemes,2,Ariquemes
7,7,NaN,RO,18 ou 19 anos,3544,1721,1823,3096,1477,1619,448,244,204,ariquemes,2,Ariquemes
8,8,NaN,RO,20 a 24 anos,8981,4456,4525,7920,3908,4012,1061,548,513,ariquemes,2,Ariquemes
9,9,NaN,RO,25 a 29 anos,8458,4161,4297,7354,3589,3765,1104,572,532,ariquemes,2,Ariquemes


In [5]:
datasetDict = {}
for i in listaUF:
  datasetDict[i] = datasetTotal.loc[(datasetTotal['UF']==i) & (datasetTotal['Idade']=='Total')]

Alguns municípios não possuem todos os dados

In [6]:
datasetTotal.loc[(datasetTotal['UF']=='CE') & (datasetTotal['ID']==947)]

,Unnamed: 0,Município_x,UF,Idade,Total_Estado,Total_Homem,Total_Mulher,Urbana_Total,Urbana_Homem,Urbana_Mulher,Rural_Total,Rural_Homem,Rural_Mulher,editMun,ID,Município_y
4715,4715,Eusébio (CE),CE,Total,13839,6814,7025,13839,6814,7025,0,0,0,eusebio,947,Eusébio
4716,4716,NaN,CE,15 a 17 anos,2871,1423,1448,2871,1423,1448,0,0,0,eusebio,947,Eusébio
4717,4717,NaN,CE,18 ou 19 anos,1750,875,875,1750,875,875,0,0,0,eusebio,947,Eusébio
4718,4718,NaN,CE,20 a 24 anos,4812,2367,2445,4812,2367,2445,0,0,0,eusebio,947,Eusébio
4719,4719,NaN,CE,25 a 29 anos,4406,2149,2257,4406,2149,2257,0,0,0,eusebio,947,Eusébio


In [7]:
nova_coluna = []
name_id = "40"     #População Jovem Rural: Total
year_id = "9"      #Ano: 2010
for index, row in datasetDict['CE'].iterrows():
  region_id = datasetDict['CE'].loc[index,'ID']
  jhi_value = datasetDict['CE'].loc[index,'Rural_Total']
  if jhi_value == 0:
    print('nulo')
  else:
    sql = "INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`)  VALUES ('NULL','" + str(region_id) + "','" + name_id + "','" + year_id + "','" + str(jhi_value) + "');"
    nova_coluna.append(sql)

nulo
nulo


Antiga função para gerar nova coluna com sqls, tratando os nulos

In [8]:
#def gerar_SQL(name, name_id, year_id, datasetUF):
#  #name = 'Rural_Total'
#  #name_id = "40"     #População Jovem Rural: Total
#  #year_id = "9"      #Ano: 2010
#  nova_coluna = []
#  string = ""
#  count = 0
#  for index, row in datasetUF.iterrows():
#    region_id = datasetUF.loc[index,'ID']
#    jhi_value = datasetUF.loc[index,name]
#    if jhi_value == "-":
#      count += 1
#    else:
#      sql = "INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`)  VALUES (NULL,'" + str(region_id) + "','" + name_id + #"','" + year_id + "','" + str(jhi_value) + "');"
#      nova_coluna.append(sql)
#
#  print("O estado " + datasetUF.loc[index,'UF'] + " tem " + str(count) + " valores nulos para o indicador " + name)
#  return nova_coluna

Nova função para gerar lista de SQLs, tratando os nulos.
Gera apenas o que vem depois do VALUES

In [9]:
def gerar_values(name, name_id, year_id, datasetUF):
    #name = 'Rural_Total'
    #name_id = "40"     #População Jovem Rural: Total
    #year_id = "9"      #Ano: 2010
    nova_coluna = []
    count = 0
    
    for index, row in datasetUF.iterrows():
        region_id = datasetUF.loc[index,'ID']
        jhi_value = datasetUF.loc[index,name]
        if jhi_value == 0:
            count += 1
        else:
            #sql = "INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`)  VALUES (NULL,'" + str(region_id) + "','" + name_id + "','" + year_id + "','" + str(jhi_value) + "');"
            sql = "(NULL,'" + str(region_id) + "','" + name_id + "','" + year_id + "','" + str(jhi_value) + "')"
            nova_coluna.append(sql)        

    print("O estado " + datasetUF.loc[index,'UF'] + " tem " + str(count) + " valores nulos para o indicador " + name)
    return nova_coluna

Nova função para gerar lista de SQL completo, aqui os nulos já estão tratados.
Gera um insert into para cada 10 values

In [10]:
def gerar_SQL(lista):
    nova_coluna = []
    auxStr = ""
    string = "INSERT INTO `MuniEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES "
    count = 1 #contador para cada 10
    
    #nova_coluna.append(string)
    auxStr += string
    
    for item in lista:
        if (count%10 == 0):
            #sql = item + "; \n"
            #nova_coluna.append(sql)
            #nova_coluna.append(string) 
            
            auxStr += item + "; "
            nova_coluna.append(auxStr) 
            auxStr = string
            
        else:
            auxStr += item + ", "
            #nova_coluna.append(sql)
        count +=1

    if auxStr.endswith(', '):
        auxStr = auxStr[:-2]
        auxStr += "; "
        nova_coluna.append(auxStr) 
    
    return nova_coluna

In [11]:
name = 'Rural_Total'
name_id = "40"     #População Jovem Rural: Total
year_id = "9"      #Ano: 2010

In [12]:
teste = gerar_values(name, name_id, year_id, datasetDict['CE'])

O estado CE tem 2 valores nulos para o indicador Rural_Total


In [12]:
teste

["(NULL,'892','40','9','1676')",
 "(NULL,'893','40','9','2108')",
 "(NULL,'894','40','9','8711')",
 "(NULL,'895','40','9','6548')",
 "(NULL,'896','40','9','3316')",
 "(NULL,'897','40','9','2147')",
 "(NULL,'898','40','9','481')",
 "(NULL,'899','40','9','2334')",
 "(NULL,'900','40','9','6703')",
 "(NULL,'901','40','9','504')",
 "(NULL,'902','40','9','2132')",
 "(NULL,'903','40','9','1835')",
 "(NULL,'904','40','9','7360')",
 "(NULL,'905','40','9','3287')",
 "(NULL,'906','40','9','1414')",
 "(NULL,'907','40','9','2310')",
 "(NULL,'908','40','9','2255')",
 "(NULL,'909','40','9','957')",
 "(NULL,'910','40','9','2616')",
 "(NULL,'911','40','9','3270')",
 "(NULL,'912','40','9','697')",
 "(NULL,'913','40','9','2194')",
 "(NULL,'914','40','9','5100')",
 "(NULL,'915','40','9','3265')",
 "(NULL,'916','40','9','2202')",
 "(NULL,'917','40','9','1383')",
 "(NULL,'918','40','9','2483')",
 "(NULL,'919','40','9','7588')",
 "(NULL,'920','40','9','5203')",
 "(NULL,'921','40','9','6656')",
 "(NULL,'922',

In [13]:
a = gerar_SQL(teste)
for i in a:
    print(i)

INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES (NULL,'892','40','9','1676'), (NULL,'893','40','9','2108'), (NULL,'894','40','9','8711'), (NULL,'895','40','9','6548'), (NULL,'896','40','9','3316'), (NULL,'897','40','9','2147'), (NULL,'898','40','9','481'), (NULL,'899','40','9','2334'), (NULL,'900','40','9','6703'), (NULL,'901','40','9','504'); 
INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES (NULL,'902','40','9','2132'), (NULL,'903','40','9','1835'), (NULL,'904','40','9','7360'), (NULL,'905','40','9','3287'), (NULL,'906','40','9','1414'), (NULL,'907','40','9','2310'), (NULL,'908','40','9','2255'), (NULL,'909','40','9','957'), (NULL,'910','40','9','2616'), (NULL,'911','40','9','3270'); 
INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES (NULL,'912','40','9','697'), (NULL,'913','40','9','2194'), (NULL,'914','40','9','5100'), (NULL,'915','40','9','32

In [16]:
listaUF

array(['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE', 'RN',
       'PB', 'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP', 'PR', 'SC',
       'RS', 'MS', 'MT', 'GO', 'DF'], dtype=object)

In [17]:
name = 'Rural_Total'
name_id = "40"     #População Jovem Rural: Total
year_id = "9"      #Ano: 2010

In [18]:
for item in listaUF:
    file_name = "Rural_Total/" + item +  "_Rural_Total.txt"
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    #text_file = open(file_name, "w")
    #for i in lista:
    #    text_file.write(i + "\n")    
    #text_file.close()

O estado RO tem 0 valores nulos para o indicador Rural_Total
O estado AC tem 0 valores nulos para o indicador Rural_Total
O estado AM tem 0 valores nulos para o indicador Rural_Total
O estado RR tem 0 valores nulos para o indicador Rural_Total
O estado PA tem 0 valores nulos para o indicador Rural_Total
O estado AP tem 0 valores nulos para o indicador Rural_Total
O estado TO tem 0 valores nulos para o indicador Rural_Total
O estado MA tem 0 valores nulos para o indicador Rural_Total
O estado PI tem 0 valores nulos para o indicador Rural_Total
O estado CE tem 2 valores nulos para o indicador Rural_Total
O estado RN tem 2 valores nulos para o indicador Rural_Total
O estado PB tem 0 valores nulos para o indicador Rural_Total
O estado PE tem 4 valores nulos para o indicador Rural_Total
O estado AL tem 0 valores nulos para o indicador Rural_Total
O estado SE tem 1 valores nulos para o indicador Rural_Total
O estado BA tem 2 valores nulos para o indicador Rural_Total
O estado MG tem 5 valore

In [19]:
file_name = "Novas/Rural_Total.sql"
text_file = open(file_name, "w")

for item in listaUF:
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    for i in lista:
        text_file.write(i + "\n")    

text_file.close()

O estado RO tem 0 valores nulos para o indicador Rural_Total
O estado AC tem 0 valores nulos para o indicador Rural_Total
O estado AM tem 0 valores nulos para o indicador Rural_Total
O estado RR tem 0 valores nulos para o indicador Rural_Total
O estado PA tem 0 valores nulos para o indicador Rural_Total
O estado AP tem 0 valores nulos para o indicador Rural_Total
O estado TO tem 0 valores nulos para o indicador Rural_Total
O estado MA tem 0 valores nulos para o indicador Rural_Total
O estado PI tem 0 valores nulos para o indicador Rural_Total
O estado CE tem 2 valores nulos para o indicador Rural_Total
O estado RN tem 2 valores nulos para o indicador Rural_Total
O estado PB tem 0 valores nulos para o indicador Rural_Total
O estado PE tem 4 valores nulos para o indicador Rural_Total
O estado AL tem 0 valores nulos para o indicador Rural_Total
O estado SE tem 1 valores nulos para o indicador Rural_Total
O estado BA tem 2 valores nulos para o indicador Rural_Total
O estado MG tem 5 valore

In [20]:
name = 'Rural_Homem'
name_id = "38"     #População Jovem Rural: Homem
year_id = "9"      #Ano: 2010

In [21]:
file_name = "Novas/Rural_Homem.sql"
text_file = open(file_name, "w")

for item in listaUF:
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    for i in lista:
        text_file.write(i + "\n")    

text_file.close()

O estado RO tem 0 valores nulos para o indicador Rural_Homem
O estado AC tem 0 valores nulos para o indicador Rural_Homem
O estado AM tem 0 valores nulos para o indicador Rural_Homem
O estado RR tem 0 valores nulos para o indicador Rural_Homem
O estado PA tem 0 valores nulos para o indicador Rural_Homem
O estado AP tem 0 valores nulos para o indicador Rural_Homem
O estado TO tem 0 valores nulos para o indicador Rural_Homem
O estado MA tem 0 valores nulos para o indicador Rural_Homem
O estado PI tem 0 valores nulos para o indicador Rural_Homem
O estado CE tem 2 valores nulos para o indicador Rural_Homem
O estado RN tem 2 valores nulos para o indicador Rural_Homem
O estado PB tem 1 valores nulos para o indicador Rural_Homem
O estado PE tem 4 valores nulos para o indicador Rural_Homem
O estado AL tem 0 valores nulos para o indicador Rural_Homem
O estado SE tem 1 valores nulos para o indicador Rural_Homem
O estado BA tem 2 valores nulos para o indicador Rural_Homem
O estado MG tem 5 valore

In [22]:
for item in listaUF:
    file_name = "Rural_Homem/" + item +  "_Rural_Homem.txt"
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    #text_file = open(file_name, "w")
    #for i in lista:
    #    text_file.write(i + "\n")    
    #text_file.close()

O estado RO tem 0 valores nulos para o indicador Rural_Homem
O estado AC tem 0 valores nulos para o indicador Rural_Homem
O estado AM tem 0 valores nulos para o indicador Rural_Homem
O estado RR tem 0 valores nulos para o indicador Rural_Homem
O estado PA tem 0 valores nulos para o indicador Rural_Homem
O estado AP tem 0 valores nulos para o indicador Rural_Homem
O estado TO tem 0 valores nulos para o indicador Rural_Homem
O estado MA tem 0 valores nulos para o indicador Rural_Homem
O estado PI tem 0 valores nulos para o indicador Rural_Homem
O estado CE tem 2 valores nulos para o indicador Rural_Homem
O estado RN tem 2 valores nulos para o indicador Rural_Homem
O estado PB tem 1 valores nulos para o indicador Rural_Homem
O estado PE tem 4 valores nulos para o indicador Rural_Homem
O estado AL tem 0 valores nulos para o indicador Rural_Homem
O estado SE tem 1 valores nulos para o indicador Rural_Homem
O estado BA tem 2 valores nulos para o indicador Rural_Homem
O estado MG tem 5 valore

In [23]:
name = 'Rural_Mulher'
name_id = "39"     #População Jovem Rural: Homem
year_id = "9"      #Ano: 2010

In [24]:
file_name = "Novas/Rural_Mulher.sql"
text_file = open(file_name, "w")

for item in listaUF:
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    for i in lista:
        text_file.write(i + "\n")    

text_file.close()

O estado RO tem 0 valores nulos para o indicador Rural_Mulher
O estado AC tem 0 valores nulos para o indicador Rural_Mulher
O estado AM tem 0 valores nulos para o indicador Rural_Mulher
O estado RR tem 0 valores nulos para o indicador Rural_Mulher
O estado PA tem 0 valores nulos para o indicador Rural_Mulher
O estado AP tem 0 valores nulos para o indicador Rural_Mulher
O estado TO tem 0 valores nulos para o indicador Rural_Mulher
O estado MA tem 0 valores nulos para o indicador Rural_Mulher
O estado PI tem 0 valores nulos para o indicador Rural_Mulher
O estado CE tem 2 valores nulos para o indicador Rural_Mulher
O estado RN tem 2 valores nulos para o indicador Rural_Mulher
O estado PB tem 0 valores nulos para o indicador Rural_Mulher
O estado PE tem 4 valores nulos para o indicador Rural_Mulher
O estado AL tem 0 valores nulos para o indicador Rural_Mulher
O estado SE tem 1 valores nulos para o indicador Rural_Mulher
O estado BA tem 2 valores nulos para o indicador Rural_Mulher
O estado

In [25]:
for item in listaUF:
    file_name = "Rural_Mulher/" + item +  "_Rural_Mulher.txt"
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    print(len(lista_values))
    lista = gerar_SQL(lista_values)

    #text_file = open(file_name, "w")
    #for i in lista:
    #    text_file.write(i + "\n")    
    #text_file.close()

O estado RO tem 0 valores nulos para o indicador Rural_Mulher
52
O estado AC tem 0 valores nulos para o indicador Rural_Mulher
22
O estado AM tem 0 valores nulos para o indicador Rural_Mulher
62
O estado RR tem 0 valores nulos para o indicador Rural_Mulher
15
O estado PA tem 0 valores nulos para o indicador Rural_Mulher
141
O estado AP tem 0 valores nulos para o indicador Rural_Mulher
16
O estado TO tem 0 valores nulos para o indicador Rural_Mulher
139
O estado MA tem 0 valores nulos para o indicador Rural_Mulher
217
O estado PI tem 0 valores nulos para o indicador Rural_Mulher
224
O estado CE tem 2 valores nulos para o indicador Rural_Mulher
181
O estado RN tem 2 valores nulos para o indicador Rural_Mulher
163
O estado PB tem 0 valores nulos para o indicador Rural_Mulher
223
O estado PE tem 4 valores nulos para o indicador Rural_Mulher
180
O estado AL tem 0 valores nulos para o indicador Rural_Mulher
102
O estado SE tem 1 valores nulos para o indicador Rural_Mulher
74
O estado BA tem 